In [7]:
%%writefile kidsandAdults_yolo_config.yaml
path: ./children_and_adults_balanced
train: images/train
val: images/val

names:
  0: kid
  1: adult

Overwriting kidsandAdults_yolo_config.yaml


In [8]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## ingat jalanin code ini dulu guys, soalnya dataset dari roboflow, labels nya terbalik

jadi fungsi code ini buat terbalikin labels nya, 0 -> kid , 1 -> adult

ingat di bagian folders diganti path nya sesuaikan dengan nama dataset roboflow yang kalian bikin


In [ ]:
import os

folders = [
    "childrenvsadults/train/labels", #path disesuaikan
    "childrenvsadults/valid/labels", #path disesuaikan
    "childrenvsadults/test/labels" #path disesuaikan
]

for folder in folders:
    for file_name in os.listdir(folder):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder, file_name)
            with open(file_path, "r") as f:
                lines = f.readlines()
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                label = parts[0]
                new_label = "1" if label == "0" else "0"  # swap 0 ↔ 1
                new_lines.append(" ".join([new_label] + parts[1:]))
            with open(file_path, "w") as f:
                f.write("\n".join(new_lines))
print("Swap label selesai di semua folder!")


Swap label selesai di semua folder!


In [8]:
import os
from collections import Counter

label_dirs = [
    "childrenvsadults/train/labels",
    "childrenvsadults/valid/labels",
    "hildrenvsadults/test/labels"
]

counts = Counter()

for labels_dir in label_dirs:
    for root, _, files in os.walk(labels_dir):
        for file in files:
            if file.endswith(".txt"):
                with open(os.path.join(root, file), "r") as f:
                    for line in f:
                        cls = int(line.split()[0])
                        counts[cls] += 1

print("Jumlah label per class setelah swap:", counts)


Jumlah label per class setelah swap: Counter({0: 1010, 1: 980})


In [29]:
import os, random, shutil

# Paths to your UTKFace datasets (adjust these if folder names differ)
src_dirs = ["archive/UTKFace", "archive/utkface_aligned_cropped"]

# Output base folder
base = "combined_dataset"
img_all = f"{base}/images/all"
lbl_all = f"{base}/labels/all"

os.makedirs(img_all, exist_ok=True)
os.makedirs(lbl_all, exist_ok=True)

# Step 1: Copy and label all images
for src in src_dirs:
 for img in os.listdir(src):
     if not img.lower().endswith(".jpg"):
         continue

     try:
         age = int(img.split("_")[0])
     except:
         continue

     # kid < 13, adult >= 13
     label = 0 if age < 13 else 1

     # copy image
     src_path = os.path.join(src, img)
     dst_img = os.path.join(img_all, img)
     shutil.copy(src_path, dst_img)

     # create YOLO label file (whole image box)
     dst_lbl = os.path.join(lbl_all, img.replace(".jpg", ".txt"))
     with open(dst_lbl, "w") as f:
         f.write(f"{label} 0.5 0.5 1 1\n")

print("✅ Copied and labeled all UTKFace images")

# Step 2: Split into train/val
os.makedirs(f"{base}/images/train", exist_ok=True)
os.makedirs(f"{base}/images/val", exist_ok=True)
os.makedirs(f"{base}/labels/train", exist_ok=True)
os.makedirs(f"{base}/labels/val", exist_ok=True)

all_imgs = [f for f in os.listdir(img_all) if f.lower().endswith(".jpg")]
random.shuffle(all_imgs)

val_split = 0.2
val_count = int(len(all_imgs) * val_split)
val_imgs = all_imgs[:val_count]
train_imgs = all_imgs[val_count:]

def move_files(img_list, split):
 for img in img_list:
     lbl = img.replace(".jpg", ".txt")
     shutil.move(f"{img_all}/{img}", f"{base}/images/{split}/{img}")
     shutil.move(f"{lbl_all}/{lbl}", f"{base}/labels/{split}/{lbl}")

move_files(train_imgs, "train")
move_files(val_imgs, "val")

print(f"✅ Train images: {len(train_imgs)}, Val images: {len(val_imgs)}")

# Step 3: Create YAML file
yaml_content = """train: combined_dataset/images/train
val: combined_dataset/images/val

names:
0: kid
1: adult
"""

with open(f"{base}/kidsAdultsCombined.yaml", "w") as f:
 f.write(yaml_content)

print("✅ YAML file created: kidsAdultsCombined.yaml")

✅ Copied and labeled all UTKFace images
✅ Train images: 18967, Val images: 4741
✅ YAML file created: kidsAdultsCombined.yaml


In [41]:
!pip uninstall torch torchvision torchaudio -y


Found existing installation: torch 2.7.1+cu118
Uninstalling torch-2.7.1+cu118:
  Successfully uninstalled torch-2.7.1+cu118
Found existing installation: torchvision 0.22.1+cu118
Uninstalling torchvision-0.22.1+cu118:
  Successfully uninstalled torchvision-0.22.1+cu118
Found existing installation: torchaudio 2.7.1+cu118
Uninstalling torchaudio-2.7.1+cu118:
  Successfully uninstalled torchaudio-2.7.1+cu118
^C


Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp310-cp310-win_amd64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp310-cp310-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp310-cp310-win_amd64.whl.metadata (6.8 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp310-cp310-win_amd64.whl (2817.2 MB)
Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp310-cp310-win_amd64.whl (5.5 MB)
Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp310-cp310-win_amd64.whl (4.1 MB)

   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp310-cp310-win_amd64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp310-cp310-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp310-cp310-win_amd64.whl.metadata (6.8 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp310-cp310-win_amd64.whl (2817.2 MB)
Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp310-cp310-win_amd64.whl (5.5 MB)
Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp310-cp310-win_amd64.whl (4.1 MB)

   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ---------------------------------------- 0/3 [torch]
   ------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
print(torch.__version__)        # harusnya keluar 2.7.1+cu118 atau mirip
print(torch.cuda.is_available()) # harus True
print(torch.cuda.get_device_name(0)) # harus NVIDIA GeForce RTX 3050


2.7.1+cu118
True
NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
# Retrain model on combined dataset
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # or your previous trained weights
results = model.train(
    data="combined_dataset/kidsAdultsCombined.yaml",
    epochs=25,
    imgsz=640,
    device=0   # gunakan GPU pertama (RTX 3050)
)



New https://pypi.org/project/ultralytics/8.3.214 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.204  Python-3.10.0 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=combined_dataset/kidsAdultsCombined.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train

In [3]:
from ultralytics import YOLO
import shutil
shutil.copy("runs/detect/train/weights/best.pt", "kdandadult-model.pt")

'kdandadult-model.pt'